# Task: Enrich your dataset with additional data from DBpedia and WikiData 


## Overview
Use the newly found DBpedia and Wikidata resources and collect additional information from DBpedia and Wikidata on the movies. It is sufficient to reuse the DBpedia and Wikidata vocabulary and use directly the DBpedia property values (including when the value is a URL/IRI), e.g., it is enough if you include the following information from DBpedia on the gross value and the producer:

<code>    
PREFIX dbo: http://dbpedia.org/ontology/
    
<https://firstname-lastname.org/resource/the_godfather>  dbo:gross    2.541E8^^xsd:double;
                                                       dbo:producer <http://dbpedia.org/page/Albert_S._Ruddy> .
<code>
    
> __Hint__: Using SPARQL’s OPTIONAL keyword might help.

## Task Details

1. Using SPARQL queries, information you __Must Get__ (if available in DBpedia and Wikidata for the movie):
    - Get the distinct genre(s) of a movie
    - Get the distinct actors 
    - Get the homepage of a movie
    - Get the number of received awards
    - Get the IMDB and the RottenTomatoes links
    - Get the box office value/gross value
    - Get the cost of a movie



## Submission 3:

Use RDFLib to load the data you have saved in Task 2 and add the additional information to the corresponding movies. As mentioned above, you can use the DBpedia and WikiData predicates. Save the enriched data set in the output folder with naming __movies_task_3.n3__.

 

<br>

## Your code

In [1]:
# Example Code
